In [1]:
import os 
import pandas as pd
from  actual_secret_key import openai_api_key
# from secret_key import openai_api_key 
from langchain.embeddings import OpenAIEmbeddings

In [2]:
os.environ['OPENAI_API_KEY'] = openai_api_key

In [3]:
embeddings_generator = OpenAIEmbeddings()

In [4]:
embeddings_generator.model

'text-embedding-ada-002'

OpenAI Model [text-embedding-ada-002](https://platform.openai.com/docs/models/embeddings) selected for good Clustering score on [Massive Text Embedding Benchmark (MTEB)](https://huggingface.co/spaces/mteb/leaderboard).